In [ ]:
import cv2
import skimage.io as io
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import helper as helper
from dataset import *
import imageio


In [ ]:
from  custom_loss import SinkhornDistance

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

n_points = 5
a = np.array([[i, 0] for i in range(n_points)])
b = np.array([[i, 1] for i in range(n_points)])

plt.figure(figsize=(6, 3))
plt.scatter(a[:, 0], a[:, 1], label='supp($p(x)$)')
plt.scatter(b[:, 0], b[:, 1], label='supp($q(x)$)')
plt.legend();


x = torch.tensor(a, dtype=torch.float)
y = torch.tensor(b, dtype=torch.float)
print(y.shape)

sinkhorn = SinkhornDistance(eps=0.1, max_iter=100)
dist, P, C = sinkhorn(x, y)
print("Sinkhorn distance: {:.3f}".format(dist.item()))


In [ ]:
label_path='../labels/coco/labels/val2017/000000007108.txt'
with open(label_path) as box:
    box=box.read()
    box=pd.DataFrame([x.split() for x in box.rstrip('\n').split('\n')],columns=['class','xc','yc','w','h'])

b= box.values.astype(np.float32)
bboxes=b
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
conf=torch.ones([b.shape[0],1])
boxes=torch.cat((b.T[1:],conf.T,one_hot_target.T)).T

img_path='../images/val2017/000000007108.jpg'
img=cv2.imread(img_path)[:,:,::-1] 

sample={'images': img,
        'boxes': boxes}

(h,w,c)=img.shape

bboxes[:,1]=bboxes[:,1]*w
bboxes[:,2]=bboxes[:,2]*h
bboxes[:,3]=bboxes[:,3]*w
bboxes[:,4]=bboxes[:,4]*h
bboxes[:,1]=bboxes[:,1]-bboxes[:,3]/2
bboxes[:,2]=bboxes[:,2]-bboxes[:,4]/2
bboxes[:,3]=bboxes[:,1]+bboxes[:,3]
bboxes[:,4]=bboxes[:,2]+bboxes[:,4]
print(bboxes)


In [ ]:
import imgaug as ia
from imgaug import parameters as iap
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


image = sample['images']

bbs = BoundingBoxesOnImage([
    BoundingBox(x1=b[1], y1=b[2], x2=b[3], y2=b[4], label=b[0]) for b in bboxes], shape=image.shape)

seq = iaa.OneOf([
            iaa.Sequential([
                iaa.LinearContrast(alpha=(0.1, 1.9)),
                iaa.Fliplr(0.5)
            ]),
            iaa.Sequential([
                iaa.Grayscale(alpha=(0.1, 0.9)),
                iaa.Affine(
                translate_px={"y": (-150, 150)}
            )
            ]),
            iaa.Sequential([
                iaa.Solarize(0.5, threshold=(0, 256)),
                iaa.ShearX((-10, 10))
            ]),
            iaa.Sequential([
                iaa.GaussianBlur(sigma=(0, 0.5)),
                iaa.ShearY((-10, 10))
            ]),
            iaa.Sequential([
                iaa.Multiply((0.8, 1.2), per_channel=0.2),
                iaa.Affine(
                rotate=(-30, 30)
            )
            ]),
            iaa.Sequential([
                iaa.HistogramEqualization(),
                iaa.Affine(
                translate_px={"x": (-150, 150)}
            )
            ]),
            iaa.Sequential([
                iaa.GaussianBlur(sigma=(0, 0.5)),
                iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}
            )
            ])
        ])

# Augment BBs and images.
image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

# print coordinates before/after augmentation (see below)
# use .x1_int, .y_int, ... to get integer coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
        i,
        before.x1, before.y1, before.x2, before.y2,
        after.x1, after.y1, after.x2, after.y2)
    )

# image with BBs before/after augmentation (shown below)
# image_before = bbs.draw_on_image(image, size=2)
# image_after = bbs_aug.draw_on_image(image_aug, size=2, color=[0, 0, 255])

In [ ]:
new_bboxes=bbs_aug.to_xyxy_array()
bbs_aug=bbs_aug.remove_out_of_image().clip_out_of_image()

ia.imshow(bbs_aug.draw_on_image(image_aug, size=2))

labels=np.array([[box.label for box in bbs_aug.bounding_boxes]]).T
new_bboxes=bbs_aug.to_xyxy_array()
np.hstack((labels,new_bboxes))

bboxes_final=helper.convert2_rel(np.hstack((labels,new_bboxes)),image_aug.shape)
print(bboxes_final)

In [ ]:
ia.imshow(bbs.draw_on_image(image, size=2))
imageio.imwrite("elephants.jpg", bbs.draw_on_image(image, size=2))
bbs_aug.draw_on_image(image_aug, size=2)
imageio.imwrite("elephants_aug.jpg", bbs_aug.draw_on_image(image_aug, size=2))

In [ ]:
box=open("../detections/000000007108.txt")
box=box.read()
box=pd.DataFrame([x.split() for x in box.rstrip('\n').split('\n')],columns=['class','conf','xc','yc','w','h'])
b= box.values.astype(np.float32)
b=torch.tensor(b)
labels = b.T[0].reshape(b.shape[0], 1)
objectness = b.T[1].reshape(b.shape[0], 1)
one_hot_target = (labels == torch.arange(80).reshape(1, 80)).float()
# conf=torch.ones([b.shape[0],1])

boxes=torch.cat((b.T[2:],objectness.T,one_hot_target.T)).T
print(boxes.shape)
# plt.imshow(draw_rect(sample['images'], boxes))
# plt.savefig('elephants.png')

In [ ]:
import cv2 
import pickle as pkl
import numpy as np 
import matplotlib.pyplot as plt

def draw_rect(im, cords, color = None):
    """Draw the rectangle on the image
    
    Parameters
    ----------
    
    im : numpy.ndarray
        numpy image 
    
    cords: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the 
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`
        
    Returns
    -------
    
    numpy.ndarray
        numpy image with bounding boxes drawn on it
        
    """
    
    im = im.copy()
    h=im.shape[0]
    w=im.shape[1]
    cords[:,0]=cords[:,0]*w
    cords[:,1]=cords[:,1]*h
    cords[:,2]=cords[:,2]*w
    cords[:,3]=cords[:,3]*h
    if not color:
        color = [255,255,255]
    for cord in cords:
        pt1, pt2 = (cord[0]-cord[2]/2, cord[1]-cord[3]/2) , (cord[0]+cord[2]/2, cord[1]+cord[3]/2)
        
#         pt1, pt2 = (cord[0], cord[1]) , (cord[2], cord[3])
        
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
    
        im = cv2.rectangle(im.copy(), pt1, pt2, color, int(max(im.shape[:2])/200))
    return im

<h1> New yolo version</h1>

In [1]:
from dataset import *
import timeit 
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import time
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import sys
import timeit
import torch.autograd
import helper as helper
import pandas as pd
from torch import autograd



net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.to(device='cuda')
cx_cy=net.cx_cy.to(device='cuda')
stride=net.stride.to(device='cuda')
print('YOLO version2')

hyperparameters={'lr': 0.001, 
                 'epochs': 90,
                 'resume_from':15,
                 'coco_version': '2017', #can be either '2014' or '2017'
                 'batch_size': 8,
                 'weight_decay': 0.0005,
                 'momentum': 0.9, 
                 'optimizer': 'sgd', 
                 'alpha': 0.5, 
                 'gamma': 0, 
                 'lcoord': 1,
                 'lno_obj': 0.5,
                 'iou_type': (0, 0, 1),
                 'iou_ignore_thresh': 0.5, 
                 'tfidf': False, 
                 'idf_weights': True, 
                 'tfidf_col_names': ['img_freq', 'none', 'none', 'none', 'no_softmax'],
                 'augment': 0, 
                 'workers': 0, 
                 'path': 'test_iou', 
                 'reduction': 'sum'}

print(hyperparameters)
coco_version=hyperparameters['coco_version']
if (hyperparameters['idf_weights']==True):
    hyperparameters['idf_weights']=pd.read_csv('../idf.csv')
else:
    hyperparameters['idf_weights']=False
        
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/'+hyperparameters['path']+'/'
    weights = torch.load(PATH+hyperparameters['path']+'_best.pth')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we https://pytorch.org/docs/stable/data.html#torch.utils.data.Datasetare on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if (torch.cuda.device_count() > 2):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
        model.load_state_dict(weights)
    else:
        model=net
        model.to(device)
        model.load_state_dict(weights)
        
except FileNotFoundError:
    try:
        os.mkdir(PATH)
    except FileExistsError:
        print('path already exist')
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if (torch.cuda.device_count() > 2):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
    else:
        model=net
        
if hyperparameters['augment']>0:
    transformed_dataset=Coco(partition='train',coco_version=coco_version,transform=transforms.Compose([Augment(hyperparameters['augment']),ResizeToTensor(inp_dim)]))
else:
    transformed_dataset=Coco(partition='train',coco_version=coco_version,transform=ResizeToTensor(inp_dim))
    


dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=hyperparameters['batch_size']
dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.collate_fn, num_workers=hyperparameters['workers'])


if hyperparameters['optimizer']=='sgd':
    optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
elif hyperparameters['otimizer']=='adam':
    optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])

scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=3)
mAP_max=0
epochs=hyperparameters['epochs']
resume_from=hyperparameters['resume_from']
break_flag=0
prg_counter=0
train_counter=0
avg_no_conf=0
avg_conf=0
avg_pos=0
avg_neg=0
total_loss=0
avg_iou=0
for images,targets in dataloader:
    train_counter=train_counter+1
    optimizer.zero_grad()
    
    images=images.cuda()
    out=model(images,'cuda')
    
    true_pred=util.transform(out.clone().detach(),pw_ph,cx_cy,stride)
    iou_list=util.get_iou_list(true_pred, targets,hyperparameters,inp_dim)
    resp_raw_pred,resp_cx_cy,resp_pw_ph,resp_stride,no_obj,w_d=util.build_tensors(out,iou_list,pw_ph,cx_cy,stride,hyperparameters)
    
    
    
    loss=util.yolo_loss(resp_raw_pred,targets,no_obj,w_d,resp_pw_ph,resp_cx_cy,resp_stride,inp_dim,hyperparameters)
    loss.backward()
    optimizer.step()
    
    stats=helper.get_progress_stats(true_pred,no_obj,iou_list,targets)
    avg_conf=avg_conf+stats['pos_conf']
    avg_no_conf=avg_no_conf+stats['neg_conf']
    avg_pos=avg_pos+stats['pos_class']
    avg_neg=avg_neg+stats['neg_class']
    total_loss=total_loss+loss.item()
    avg_iou=avg_iou+stats['iou']
    prg_counter=prg_counter+1
    
    sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%' ' L:'+ str(loss.item()))
    sys.stdout.write(' IoU:' +str(stats['iou'])+' pob:'+str(stats['pos_conf'])+ ' nob:'+str(stats['neg_conf']))
    sys.stdout.write(' PCls:' +str(stats['pos_class'])+' ncls:'+str(stats['neg_class']))
    sys.stdout.flush()
    
    
    
    

YOLO version2
{'lr': 0.001, 'epochs': 90, 'resume_from': 15, 'coco_version': '2017', 'batch_size': 8, 'weight_decay': 0.0005, 'momentum': 0.9, 'optimizer': 'sgd', 'alpha': 0.5, 'gamma': 0, 'lcoord': 1, 'lno_obj': 0.5, 'iou_type': (0, 0, 1), 'iou_ignore_thresh': 0.5, 'tfidf': False, 'idf_weights': True, 'tfidf_col_names': ['img_freq', 'none', 'none', 'none', 'no_softmax'], 'augment': 0, 'workers': 0, 'path': 'test_iou', 'reduction': 'sum'}
path already exist
cuda:0
Length of dataset is 117266

Pgr:5.171149352753568% L:0.6401216983795166 IoU:0.7320887446403503 pob:0.48008647561073303 nob:0.0068601686507463455 PCls:0.9185643792152405 ncls:0.0010308304335922003123

KeyboardInterrupt: 

In [5]:
import test
mAP=test.evaluate(model, device,coco_version,confidence=0.01,iou_threshold=0.5)

creating index...
index created!
Accumulating evaluation results...
DONE (t=11.78s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.509
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.218
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.295
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.205
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.346
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.383
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.242
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.390
 Average Recall   

<h1>Wasserstein</h1>

In [2]:
min_dim=inp_dim//32
tt=[]
dd=[]
for j in range(len(iou_list)):
    distribution=(iou_list[j].max(axis=0))[0]
    for i in range(3):
        slce=min_dim*(2**i)

        ind=slce*slce*3
        prev=0


        tt.append(true_pred[0,prev:prev+ind,4].reshape(3,slce,slce).cpu())
        dd.append(distribution[prev:prev+ind].reshape(3,slce,slce).cpu())
        prev=ind

In [ ]:
print(len(tt))

In [3]:
from  custom_loss import SinkhornDistance



sinkhorn = SinkhornDistance(eps=0.1, max_iter=100,reduction='sum')
total_dist=0
for i in range(len(tt)):
    dist, P, C = sinkhorn(dd[i], tt[i])
    total_dist+=dist
    
print(total_dist/8)
    


RuntimeError: expected device cpu but got device cuda:0

In [ ]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
import json

net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.cuda()
cx_cy=net.cx_cy.cuda()
stride=net.stride.cuda()

start_time = time.time()
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/pretrained32_precomp_obj_soft.pth'
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if torch.cuda.device_count() > 1:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
    else:
        model=net

    model.to(device)
    
    
    model.load_state_dict(weights)
    model.eval()
except FileNotFoundError: 
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    model=net
    if torch.cuda.device_count() > 3:
      print("Using ", torch.cuda.device_count(), "GPUs!")
      model = nn.DataParallel(net)

    model.to(device)
    model.eval()




print('testing with '+ PATH +'\n')
transformed_dataset=Coco(partition='val',coco_version='2014',subset=0.1,
                                           transform=transforms.Compose([
                                            ResizeToTensor(inp_dim)
                                           ]))



dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=2

dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.my_collate, num_workers=4)

true_pos=0
false_pos=0
counter=0
iou_threshold=0.5
confidence=0.01
recall_counter=0
max_detections=None
prg_counter=0
annotations=[]
for images,targets,img_meta in dataloader:
#     print(images)
    inp=images.cuda()
    raw_pred = model(inp, torch.cuda.is_available())
    targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)

    raw_pred=raw_pred.to(device='cuda')
    true_pred=util.transform(raw_pred.clone(),pw_ph,cx_cy,stride)

    sorted_pred=torch.sort(true_pred[:,:,4],descending=True)
    pred_mask=sorted_pred[0]>confidence
    indices=[(sorted_pred[1][e,:][pred_mask[e,:]]) for e in range(pred_mask.shape[0])]
    pred_final=[true_pred[i,indices[i],:] for i in range(len(indices))]

    pred_final_coord=[util.get_abs_coord(pred_final[i].unsqueeze(-2)) for i in range(len(pred_final))]

    indices=[nms_box.nms(pred_final_coord[i][0],pred_final[i][:,4],iou_threshold) for i in range(len(pred_final))]

    pred_final=[pred_final[i][indices[i],:] for i in range(len(pred_final))]


    abs_pred_final=[helper.convert2_abs_xywh(pred_final[i],img_meta[i]['img_size'],inp_dim) for i in range(len(pred_final))]
    
    annotations=[*annotations,*helper.transform_to_COCO(img_meta,abs_pred_final)]
    sys.stdout.write('\rPgr:'+str(prg_counter/dataset_len*100*batch_size)+'%')
    prg_counter=prg_counter+1
    
    

print("--- %s seconds ---" % (time.time() - start_time))
with open('data.json', 'w') as f:
    json.dump(annotations, f)
# Read txt files containing bounding boxes (ground truth and detections)


In [ ]:
import json

with open('data.json', 'w') as f:
    json.dump(annotations, f)



In [ ]:
annotations[0]

In [ ]:
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import pandas as pd
import time
import sys
import timeit
from dataset import *
import torchvision.ops.boxes as nms_box
import helper as helper
import test as tester
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
import json

net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.cuda()
cx_cy=net.cx_cy.cuda()
stride=net.stride.cuda()

start_time = time.time()
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/yolo201sds7/yolo2017_best.pth'
    weights = torch.load(PATH)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if torch.cuda.device_count() > 2:
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
    else:
        model=net

    model.to(device)
    
    
    model.load_state_dict(weights)
    model.eval()
except FileNotFoundError: 
    net.load_weights("../yolov3.weights")
    print('orginal weights loaded')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net.to(device)
    

    
net.train()

subset=0.0006
coco_version='2014'
confidence=0.1
iou_threshold=0.5


max_detections=100
transformed_dataset=Coco(partition='val',coco_version=coco_version,subset=subset,
                                           transform=transforms.Compose([
                                            ResizeToTensor(inp_dim)
                                           ]))



dataset_len=(len(transformed_dataset))
# print(dataset_len)
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=8

dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=False,collate_fn=helper.my_collate, num_workers=4)


for images,targets,img_name in dataloader:
    inp=images.cuda()
    raw_pred = net(inp, torch.cuda.is_available())
    targets,anchors,offset,strd,mask=helper.collapse_boxes(targets,pw_ph,cx_cy,stride)

    raw_pred=raw_pred.to(device='cuda')
    true_pred=util.transform(raw_pred.clone(),pw_ph,cx_cy,stride)
    sorted_pred=torch.sort(true_pred[:,:,4]*(true_pred[:,:,5:].max(axis=2)[0]),descending=True)
    pred_mask=sorted_pred[0]>confidence
    indices=[(sorted_pred[1][e,:][pred_mask[e,:]]) for e in range(pred_mask.shape[0])]
    pred_final=[true_pred[i,indices[i],:] for i in range(len(indices))]
    pred_final_coord=[util.get_abs_coord(pred_final[i].unsqueeze(-2)) for i in range(len(pred_final))]
    
    indices=[nms_box.nms(pred_final_coord[i][0],pred_final[i][:,4],iou_threshold) for i in range(len(pred_final))]

    pred_final=[pred_final[i][indices[i],:] for i in range(len(pred_final))]
    
    abs_pred_final=[helper.convert2_abs_xywh(pred_final[i],img_name[i]['img_size'],inp_dim) for i in range(len(pred_final))]
    
    break
    
#     pred_final[:,0:4]=pred_final[:,0:4]/inp_dim
    helper.write_pred(img_name,pred_final,inp_dim,max_detections,coco_version)



a=tester.evaluate(net, 'cpu',coco_version='2014',confidence=0.1,iou_threshold=0.5,subset=0.0003)

# a=tester.get_map(model, confidence=0.1,iou_threshold=0.5,coco_version='2014',subset=0.0003)


# print(a)
# subset=0.0003

In [ ]:
a=tester.evaluate(net, 'cpu',coco_version='2014',confidence=0.01,iou_threshold=0.5,subset=0.1)

In [ ]:
print(targets.shape)
targets_list=[targets[:13,:],targets[13:,:]]
abs_targets=[helper.convert2_abs_xywh(targets_list[i],img_name[i]['img_size'],1) for i in range(len(targets_list))]
print(abs_targets)

<h1>More efficient Yolo</h1>

In [1]:
from dataset import *
import timeit 
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import time
from darknet import *
import util as util
import torch.optim as optim
import test as tester
import sys
import timeit
import torch.autograd
import helper as helper
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import yolo_function as yolo_function


hyperparameters={'lr': 0.0001, 
                 'epochs': 90,
                 'resume_from':0,
                 'coco_version': '2017', #can be either '2014' or '2017'
                 'batch_size': 8,
                 'weight_decay': 0.0005,
                 'momentum': 0.9, 
                 'optimizer': 'sgd', 
                 'alpha': 0.9, 
                 'gamma': 0, 
                 'lcoord': 5,
                 'lno_obj': 0.5,
                 'iou_type': (1, 0, 0),
                 'iou_ignore_thresh': 0.213, 
                 'tfidf': True, 
                 'idf_weights': True, 
                 'tfidf_col_names': ['img_freq', 'none', 'none', 'none', 'no_softmax'],
                 'inf_confidence':0.01,
                 'inf_iou_threshold':0.5,
                 'augment': 1, 
                 'workers': 4,
                 'pretrained':False,
                 'path': 'yolo2017', 
                 'reduction': 'sum'}

mode={'bayes_opt':False,
      'debugging':False,
      'show_output':True,
      'multi_gpu':False,
      'show_temp_summary':False,
      'save_summary': True
     }

print(hyperparameters)
if isinstance(hyperparameters['idf_weights'],pd.DataFrame)==False:
    if (hyperparameters['idf_weights']==True):
        hyperparameters['idf_weights']=pd.read_csv('../idf.csv')
    else:
        hyperparameters['idf_weights']=False
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using: ',device)

net = Darknet("../cfg/yolov3.cfg")
coco_version=hyperparameters['coco_version']
inp_dim=net.inp_dim

'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/'+hyperparameters['path']+'/'
    checkpoint = torch.load(PATH+hyperparameters['path']+'.tar')
                    # Assuming that we https://pytorch.org/docs/stable/data.html#torch.utils.data.Datasetare on a CUDA machine, this should print a CUDA device:
    net.to(device)

    if (torch.cuda.device_count() > 1)&(mode['multi_gpu']==True):
        model = nn.DataParallel(net)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
    else:
        model=net
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)

    if hyperparameters['optimizer']=='sgd':
        optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
    elif hyperparameters['optimizer']=='adam':
        optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    hyperparameters['resume_from']=checkpoint['epoch']

except FileNotFoundError:
    if (hyperparameters['pretrained']==True):
        print("WARNING FILE NOT FOUND INSTEAD USING OFFICIAL PRETRAINED")
        net.load_weights("../yolov3.weights")

        net.to(device)
        if (torch.cuda.device_count() > 1)&(mode['multi_gpu']==True):
            model = nn.DataParallel(net)
            model.to(device)
        else:
            model=net

        if hyperparameters['optimizer']=='sgd':
            optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
        elif hyperparameters['optimizer']=='adam':
            optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])
        hyperparameters['resume_from']=0
    else:
        try:
            PATH = '../pth/'+hyperparameters['path']+'/'
            os.mkdir(PATH)
        except FileExistsError:
            pass
#                     print('path already exist')

        if (torch.cuda.device_count() > 1)&(mode['multi_gpu']==True):
            model = nn.DataParallel(net)
            model.to(device)
        else:
            model=net
            model.to(device)

        if hyperparameters['optimizer']=='sgd':
            optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
        elif hyperparameters['optimizer']=='adam':
            optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])
        hyperparameters['resume_from']=0
    
    
if(mode['show_temp_summary']==True):
    writer = SummaryWriter('../results/test_vis/')



if hyperparameters['augment']>0:
    train_dataset=Coco(partition='train',coco_version=coco_version,transform=transforms.Compose([Augment(hyperparameters['augment']),ResizeToTensor(inp_dim)]))
else:
    train_dataset=Coco(partition='train',coco_version=coco_version,transform=transforms.Compose([ResizeToTensor(inp_dim)]))
    
dataset_len=(len(train_dataset))
batch_size=hyperparameters['batch_size']
mAP_best=0

scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=5)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True,collate_fn=helper.collate_fn, num_workers=hyperparameters['workers'])


for i in range(hyperparameters['epochs']):
    outcome=yolo_function.train_one_epoch(model,optimizer,train_dataloader,hyperparameters,mode)
    mAP=test.evaluate(model, device,coco_version,confidence=hyperparameters['inf_confidence'],iou_threshold=hyperparameters['inf_iou_threshold'])
    scheduler.step(mAP)
    if(mode['save_summary']==True):
        writer = SummaryWriter('../results/'+hyperparameters['path'])
        writer.add_scalar('Loss/train', outcome['avg_loss'], hyperparameters['resume_from'])
        writer.add_scalar('AIoU/train', outcome['avg_iou'], hyperparameters['resume_from'])
        writer.add_scalar('PConf/train', outcome['avg_conf'], hyperparameters['resume_from'])
        writer.add_scalar('NConf/train', outcome['avg_no_conf'], hyperparameters['resume_from'])
        writer.add_scalar('PClass/train', outcome['avg_pos'], hyperparameters['resume_from'])
        writer.add_scalar('NClass/train', outcome['avg_neg'], hyperparameters['resume_from'])
        
        writer.add_scalar('mAP/valid', mAP, hyperparameters['resume_from'])

    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'avg_loss': outcome['total_loss'],
            'avg_iou': outcome['avg_iou'],
            'avg_pos': outcome['avg_pos'],
            'avg_neg':outcome['avg_neg'],
            'avg_conf': outcome['avg_conf'],
            'avg_no_conf': outcome['avg_no_conf'],
            'epoch':hyperparameters['resume_from']+1,
            'mAP': mAP
            }, PATH+hyperparameters['path']+'.tar')

        
    if mAP>mAP_best:
        torch.save(checkpoint, PATH+hyperparameters['path']+'_best.tar')
        mAP_best=mAP
        
    

{'lr': 0.0001, 'epochs': 90, 'resume_from': 0, 'coco_version': '2017', 'batch_size': 8, 'weight_decay': 0.0005, 'momentum': 0.9, 'optimizer': 'sgd', 'alpha': 0.9, 'gamma': 0, 'lcoord': 5, 'lno_obj': 0.5, 'iou_type': (1, 0, 0), 'iou_ignore_thresh': 0.213, 'tfidf': True, 'idf_weights': True, 'tfidf_col_names': ['img_freq', 'none', 'none', 'none', 'no_softmax'], 'inf_confidence': 0.01, 'inf_iou_threshold': 0.5, 'augment': 1, 'workers': 4, 'pretrained': False, 'path': 'yolo2017', 'reduction': 'sum'}
Using:  cuda:0
Pgr:0.8186516125731242% L:7.840477466583252 IoU:0.7468103170394897 pob:0.2061162143945694 nob:0.006857478525489569 PCls:0.5595539808273315 ncls:0.00557526666671037795755

KeyboardInterrupt: 

In [ ]:
print(checkpoint['epoch'])

In [ ]:
torch.save({
'model_state_dict': model.state_dict(),
'optimizer_state_dict': optimizer.state_dict(),
'avg_loss': 0,
'avg_iou': 0,
'avg_pos': 0,
'avg_neg':0,
'avg_conf': 0,
'avg_no_conf': 0,
'epoch':hyperparameters['resume_from']
}, PATH+hyperparameters['path']+'.tar')

In [ ]:
hyperparameters['idf_weights']=True

In [ ]:
from dataset import *
import timeit 
import cv2
import numpy as np
from zipfile import ZipFile
import pandas as pd
import torch
import time
from darknet import *
import darknet as dn
import util as util
import torch.optim as optim
import sys
import timeit
import torch.autograd
import helper as helper
import pandas as pd
from torch import autograd
import test as tester
import yolo_function



net = Darknet("../cfg/yolov3.cfg")
inp_dim=net.inp_dim
pw_ph=net.pw_ph.to(device='cuda')
cx_cy=net.cx_cy.to(device='cuda')
stride=net.stride.to(device='cuda')
print('YOLO version2')

hyperparameters={'lr': 0.00001, 
                 'epochs': 90,
                 'resume_from':15,
                 'coco_version': '2017', #can be either '2014' or '2017'
                 'batch_size': 8,
                 'weight_decay': 0.0005,
                 'momentum': 0.9, 
                 'optimizer': 'sgd', 
                 'alpha': 0.5, 
                 'gamma': 0, 
                 'lcoord': 1,
                 'lno_obj': 0.5,
                 'iou_type': (1, 0, 0),
                 'iou_ignore_thresh': 0.5, 
                 'tfidf': False, 
                 'idf_weights': True, 
                 'tfidf_col_names': ['img_freq', 'none', 'none', 'none', 'no_softmax'],
                 'inf_confidence':0.01,
                 'inf_iou_threshold':0.5,
                 'augment': 0, 
                 'workers': 0, 
                 'path': 'yolo2017', 
                 'reduction': 'sum'}


bayes_opt=True
mode={'bayes_opt':bayes_opt,
      'debugging':False,
      'show_output':True,
      'multi_gpu':False,
      'show_temp_summary':False,
      'save_summary': bayes_opt==False
     }

print(hyperparameters)
coco_version=hyperparameters['coco_version']
if (hyperparameters['idf_weights']==True):
    hyperparameters['idf_weights']=pd.read_csv('../idf.csv')
else:
    hyperparameters['idf_weights']=False
        
'''
when loading weights from dataparallel model then, you first need to instatiate the dataparallel model 
if you start fresh then first model.load_weights and then make it parallel
'''
try:
    PATH = '../pth/'+hyperparameters['path']+'/'
    weights = torch.load(PATH+hyperparameters['path']+'_best.pth')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we https://pytorch.org/docs/stable/data.html#torch.utils.data.Datasetare on a CUDA machine, this should print a CUDA device:
    print(device)
    net.to(device)

    if (torch.cuda.device_count() > 2):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
        model.load_state_dict(weights)
    else:
        model=net
        model.to(device)
        model.load_state_dict(weights)
        
except FileNotFoundError:
    try:
        os.mkdir(PATH)
    except FileExistsError:
        print('path already exist')
    net.load_weights("../yolov3.weights")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Assuming that we are on a CUDA machine, this should print a CUDA device:

    print(device)
    net.to(device)
    if (torch.cuda.device_count() > 2):
        print("Using ", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(net)
        model.to(device)
    else:
        model=net
        
if hyperparameters['augment']>0:
    transformed_dataset=Coco(partition='train',coco_version=coco_version,subset=0.001,transform=transforms.Compose([Augment(hyperparameters['augment']),ResizeToTensor(inp_dim)]))
else:
    transformed_dataset=Coco(partition='train',coco_version=coco_version,subset=0.001,transform=ResizeToTensor(inp_dim))
    


dataset_len=(len(transformed_dataset))
print('Length of dataset is '+ str(dataset_len)+'\n')
batch_size=hyperparameters['batch_size']
dataloader = DataLoader(transformed_dataset, batch_size=batch_size,
                        shuffle=True,collate_fn=helper.collate_fn, num_workers=hyperparameters['workers'])


if hyperparameters['optimizer']=='sgd':
    optimizer = optim.SGD(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'], momentum=hyperparameters['momentum'])
elif hyperparameters['otimizer']=='adam':
    optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])

    
outcome=yolo_function.train_one_epoch(model,optimizer,dataloader,hyperparameters,mode)
tester.evaluate(model, device,coco_version,confidence=0.01,iou_threshold=0.5,subset=0.01)



    

In [ ]:
print(optimizer)